# My Data Selection
I have chosen to research Electric Vehicle Population Data collected in the state of Washington. This dataset shows the Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through Washington State Department of Licensing (DOL).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
evData = pd.read_csv("..\capstone_1\Electric_Vehicle_Population_Data.csv")

In [3]:
evData.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5UXTA6C03P,King,Seattle,WA,98177.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,36.0,218985539,POINT (-122.38242499999996 47.77279000000004),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
1,1FMCU0EZXN,Yakima,Moxee,WA,98936.0,2022,FORD,ESCAPE,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,15.0,197264322,POINT (-120.37951169999997 46.55609000000004),PACIFICORP,5.307700e+10
2,1G1FW6S03J,King,Seattle,WA,98117.0,2018,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,36.0,168549727,POINT (-122.37275999999997 47.689685000000054),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,5YJSA1AC0D,King,Newcastle,WA,98059.0,2013,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208,69900,41.0,244891062,POINT (-122.15733999999998 47.487175000000036),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,1FADP5CU8F,Kitsap,Bremerton,WA,98312.0,2015,FORD,C-MAX,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,19,0,26.0,134915000,POINT (-122.65223 47.57192),PUGET SOUND ENERGY INC,5.303508e+10


# My hypothesis about the data:
### Electric vehicles in the state of Washington are primarily driven by people who live in Seattle.

and now testing that hypothesis...

In [4]:
# I want to know if the EV is registered in Seattle or not.

def seattle_true(city):
    if city != 'Seattle':
        return False
    else:
        return True
    
evData['in_seattle'] = evData['City'].apply(seattle_true)
seaData = evData.sort_values('in_seattle', ascending = False)[['in_seattle','City','State','Make','Model']]
pd.DataFrame(seaData)
seaData

,in_seattle,City,State,Make,Model
0,True,Seattle,WA,BMW,X5
19602,True,Seattle,WA,AUDI,Q5 E
113736,True,Seattle,WA,HYUNDAI,KONA ELECTRIC
127722,True,Seattle,WA,TESLA,MODEL Y
93179,True,Seattle,WA,TOYOTA,PRIUS PRIME
...,...,...,...,...,...
53893,False,Bothell,WA,NISSAN,LEAF
53892,False,Bothell,WA,TESLA,MODEL Y
53891,False,Seatac,WA,CHEVROLET,BOLT EV
53888,False,Kent,WA,TESLA,MODEL 3


In [5]:
# First I want to know if there are more EVs in Seattle than in the rest of Washington (just Washington):

sea_ev = seaData.in_seattle.mode().loc[seaData['State'] == 'WA']
print(f'{sea_ev} \n')
if sea_ev.bool() != True:
    print('There are not more electric vehicles in Seattle than the rest of the state of Washington.')

0    False
Name: in_seattle, dtype: bool 

There are not more electric vehicles in Seattle than the rest of the state of Washington.


In [6]:
# Now I want to see all of the cities with registered ev's

drop_evData = evData.drop_duplicates('City').loc[evData['State'] == 'WA'].sort_values('City').dropna()[['City','State']]
drop_evData

,City,State
3478,Aberdeen,WA
96414,Acme,WA
101125,Addy,WA
62093,Airway Heights,WA
112343,Alderdale,WA
...,...,...
364,Yacolt,WA
8,Yakima,WA
32637,Yarrow Point,WA
40,Yelm,WA


In [7]:
# 456 cities is a lot to go through, maybe lets group them by county. Seattle is in King County.

evData_sorted = evData.sort_values(['County','City']).loc[seaData['State'] == 'WA'][['County','City','State','Make','Model']]

# let's exclude NULL city and county data for now..
ev_county_notNull = evData_sorted.dropna()
display(ev_county_notNull)

# Now the most common county
sort_county = ev_county_notNull.County.mode()
print(f'The county with the most registered electric vehicles is:\n{sort_county}')

,County,City,State,Make,Model
85918,Adams,Hatton,WA,TESLA,MODEL 3
74836,Adams,Lind,WA,CHRYSLER,PACIFICA
87486,Adams,Lind,WA,RIVIAN,R1T
59114,Adams,Othello,WA,CHEVROLET,VOLT
59432,Adams,Othello,WA,TESLA,MODEL 3
...,...,...,...,...,...
74617,Yakima,Zillah,WA,BMW,X5
101666,Yakima,Zillah,WA,CHEVROLET,BOLT EUV
108575,Yakima,Zillah,WA,KIA,EV6
117730,Yakima,Zillah,WA,TESLA,MODEL 3


The county with the most registered electric vehicles is:
0    King
Name: County, dtype: object


### The county (in Washington State) with the most registered electric vehicles is King county. Now I want to know if Seattle is the city within King County with the most electric vehicles, and that will show me if my hypothesis is correct.

In [8]:
# Lookin good for my hypothesis...
# All of the counties now maybe...

drop_county = ev_county_notNull.drop_duplicates('County').sort_values('County').dropna()[['County','City']]
display(drop_county)

,County,City
85918,Adams,Hatton
85621,Asotin,Asotin
60093,Benton,Benton City
10268,Chelan,Cashmere
89472,Clallam,Beaver
7455,Clark,Amboy
69749,Columbia,Dayton
5603,Cowlitz,Ariel
140800,Douglas,Bridgeport
95097,Ferry,Curlew


In [9]:
# Cities in King county. All data sorted by city
evCountySort = evData_sorted.sort_values('City').loc[evData_sorted['County'] == 'King'][['County','City','Make','Model']]
# Only the cities in King county
onlyKingCities = evCountySort.drop_duplicates('City').sort_values('City')

# Now the most common city
sort_city = evCountySort.City.mode()
print(f'The city with the most registered electric vehicles in King county is:\n{sort_city}')

The city with the most registered electric vehicles in King county is:
0    Seattle
Name: City, dtype: object


## In the state of Washington, Seattle has the most registered electric vehicles!  This proves my hypothesis is correct.
Let's find a simpler way to reach that conclusion.

In [10]:
seaSort = evData.sort_values('City').loc[evData['State'] == 'WA'].dropna()[['City','VIN (1-10)']]
seaCount = seaSort['City'].value_counts()
display(seaCount.head())

Seattle      24662
Bellevue      7375
Redmond       5245
Vancouver     5045
Bothell       4599
Name: City, dtype: int64

#### This also proves my hypothesis to be true.

In [11]:
seaCount.nlargest(n=1, keep='first')

Seattle    24662
Name: City, dtype: int64

# Further Data Research:
### Knowing that most electric vehicles in Washington state are registered in Seattle is all well and good, but lets explore more!

## Which EV make and model is the most popular?
### What are the top 5 most popular EVs?

In [12]:
makeModel = evData.sort_values('Make')[['Make','Model']]
makeModelCount = makeModel.value_counts()
makeModelCount.head(5)

Make       Model  
TESLA      MODEL 3    26766
           MODEL Y    26194
NISSAN     LEAF       13093
TESLA      MODEL S     7542
CHEVROLET  BOLT EV     5599
dtype: int64

### The TESLA MODEL 3 is the most popular EV, followed closely by the TESLA MODEL Y.

### Which is the LEAST popular make and model?

In [13]:
makeModelCount.tail(5)

Make       Model      
BENTLEY    BENTAYGA       2
AUDI       A8 E           2
PORSCHE    918            1
CHEVROLET  S-10 PICKUP    1
BENTLEY    FLYING SPUR    1
dtype: int64

### It's a tie for last! The Porsche 918, Chevrolet S-10 Pickup, and Bentley Flying Spur are the least popular EVs.

## Let's see whether Battery Electric Vehicles (BEVs) or Plug-in Hybrid Electric Vehicles (PHEVs) are more popular.

In [14]:
evType = evData.sort_values('Electric Vehicle Type')[['Electric Vehicle Type','Model Year','Make','Model']]

def type_group(type):
    if type != 'Battery Electric Vehicle (BEV)':
        return 'Hybrid'
    else:
        return 'Battery'
    
evType['Type Group'] = evType['Electric Vehicle Type'].apply(type_group)
typeData = evType.sort_values('Type Group')[['Type Group','Model Year','Make','Model']]
typeData.value_counts('Type Group')

Type Group
Battery    110865
Hybrid      32731
dtype: int64

#### The BEVs are definitely more popular.

## How many of the BEVs are eligible as a Clean Alternative Fuel Vehicle?

In [15]:
evData['Type Group'] = evData['Electric Vehicle Type'].apply(type_group)
cafv_elig = evData.sort_values('Type Group').loc[evData['Clean Alternative Fuel Vehicle (CAFV) Eligibility']
                                                 == 'Clean Alternative Fuel Vehicle Eligible'][['Type Group','Clean Alternative Fuel Vehicle (CAFV) Eligibility']]
display(cafv_elig.value_counts())

cafv_not_elig = cafv_elig = evData.sort_values('Type Group').loc[evData['Clean Alternative Fuel Vehicle (CAFV) Eligibility']
                                                 == 'Not eligible due to low battery range'][['Type Group','Clean Alternative Fuel Vehicle (CAFV) Eligibility']]
display(cafv_not_elig.value_counts())

Type Group  Clean Alternative Fuel Vehicle (CAFV) Eligibility
Battery     Clean Alternative Fuel Vehicle Eligible              46903
Hybrid      Clean Alternative Fuel Vehicle Eligible              15246
dtype: int64

Type Group  Clean Alternative Fuel Vehicle (CAFV) Eligibility
Hybrid      Not eligible due to low battery range                17485
Battery     Not eligible due to low battery range                    8
dtype: int64

## What is the average electric range of a BEV vurses a PHEV?

In [16]:
rangeEV = evData.sort_values(['Type Group','Electric Range'])[['Type Group','Make','Electric Range']]

# Lets remove the null (or 0) values for now.
rangeEV_drop = rangeEV.drop(rangeEV[rangeEV['Electric Range'] == 0].index)

BEV = rangeEV_drop.sort_values('Electric Range').loc[rangeEV_drop['Type Group'] == 'Battery']['Electric Range']
BEV_ave = int(BEV.mean())
print(f'The average electric range of a Battery Electric Vehicle (BEV) is {round(BEV_ave)} miles.')

PHEV = rangeEV_drop.sort_values('Electric Range').loc[rangeEV_drop['Type Group'] == 'Hybrid']['Electric Range']
PHEV_ave = int(PHEV.mean())
print(f'The average electric range of a Plug-in Hybrid Electric Vehicle (PHEV) is {round(PHEV_ave)} miles.\n')

BEV_max = int(BEV.max())
BEV_min = int(BEV.min())
PHEV_max = int(PHEV.max())
PHEV_min = int(PHEV.min())

print(f'The top electric range of a BEV is {round(BEV_max)}, and the lowest range for a BEV is {round(BEV_min)}.')
print(f'The top electric range of a PHEV is {round(PHEV_max)}, and the lowest range is {round(PHEV_min)}.')

The average electric range of a Battery Electric Vehicle (BEV) is 194 miles.
The average electric range of a Plug-in Hybrid Electric Vehicle (PHEV) is 30 miles.

The top electric range of a BEV is 337, and the lowest range for a BEV is 29.
The top electric range of a PHEV is 153, and the lowest range is 6.
